In [43]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


In [2]:
class ScriptData:

    def __init__(self):
        self.data = {}
        self.ts = TimeSeries(key='MQEA4163N87R0BIO')
        

    def fetch_intraday_data(self,sym):
        df, meta_data = self.ts.get_intraday(sym)
        self.data[f'{sym}'] = df

        return df , sym
    def convert_intraday_data(self,sym):
        df = self.data[sym]
        df = pd.DataFrame(df).transpose().reset_index()
        df.columns = ['timestamp','open','high','low','close','volume']
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['volume'] = pd.to_numeric(df['volume'])
        df['close'] = pd.to_numeric(df['close'])
        df['high'] = pd.to_numeric(df['high'])
        df['low'] = pd.to_numeric(df['low'])
        df['open'] = pd.to_numeric(df['open'])
        self.data[f'{sym}'] = df

        return df

    def __getitem__(self, key):
          
        return self.data[key]
    

def indicator1(df , timeperiod):

    df['indicator'] = np.nan

    for i in range(timeperiod-1, len(df)):
        df['indicator'][i] = sum(df['close'][i-timeperiod+1 : i+1])/timeperiod

    return df[['timestamp','indicator']].copy()




In [3]:
script = ScriptData()

In [24]:
script.fetch_intraday_data('GOOGL')
df = script.convert_intraday_data('GOOGL')
df


,timestamp,open,high,low,close,volume
0,2022-12-27 20:00:00,87.3800,87.380,87.180,87.3599,5168
1,2022-12-27 19:45:00,87.2700,87.370,87.190,87.3700,1917
2,2022-12-27 19:30:00,87.3900,87.390,87.280,87.2800,1711
3,2022-12-27 19:15:00,87.3000,87.490,87.300,87.3900,3361
4,2022-12-27 19:00:00,87.3500,87.370,87.320,87.3700,1001
...,...,...,...,...,...,...
95,2022-12-23 12:00:00,88.9850,89.120,88.800,89.1200,510227
96,2022-12-23 11:45:00,88.6900,89.195,88.565,88.9949,459756
97,2022-12-23 11:30:00,88.5803,88.900,88.520,88.6850,608166
98,2022-12-23 11:15:00,88.7700,88.890,88.210,88.5800,822376


In [25]:

indicator1(df, timeperiod= 5)

,timestamp,indicator
0,2022-12-27 20:00:00,NaN
1,2022-12-27 19:45:00,NaN
2,2022-12-27 19:30:00,NaN
3,2022-12-27 19:15:00,NaN
4,2022-12-27 19:00:00,87.35398
...,...,...
95,2022-12-23 12:00:00,89.15500
96,2022-12-23 11:45:00,89.12098
97,2022-12-23 11:30:00,89.02398
98,2022-12-23 11:15:00,88.89798


In [80]:
class Strategy:

    def __init__(self , symbol):
        self.symbol  = symbol
        

        self.df = pd.DataFrame()

    def get_script_data(self):
        self.script = ScriptData()
        self.script.fetch_intraday_data(self.symbol)
        self.script.convert_intraday_data(self.symbol)

        self.df = self.script.data[self.symbol]

        self.indicator = indicator1(df , timeperiod=5)

        self.df['indicator_data'] = self.indicator['indicator']


    def get_signals(self):
        df = self.df

        df['signal'] = np.nan
        for i in range(len(df)):
            if df.iloc[i-1]['close']>df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] < df.iloc[i]['indicator_data']:
                df['signal'][i] = 'BUY'
            elif df.iloc[i-1]['close'] < df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] > df.iloc[i]['indicator_data']:
                df['signal'][i] = 'SELL'

        self.df = df
        

        return self.df[['timestamp','signal']].copy().dropna()

    def plot(self):
        df = self.df
        fig = px.line(df , y = ['close','indicator_data'])
        fig.show()

        fig = go.Figure(data=[go.Candlestick(
                       open=df['open'], high=df['high'],
                       low=df['low'], close=df['close'] ) ])
        fig.add_trace(go.Line(y=df['indicator_data']))

        fig.show()



    

In [81]:
strategy = Strategy('GOOGL')

In [82]:
strategy.get_script_data()

In [83]:
strategy.get_signals()

,timestamp,signal
5,2022-12-27 18:45:00,BUY
6,2022-12-27 18:30:00,SELL
8,2022-12-27 18:00:00,BUY
13,2022-12-27 16:45:00,SELL
19,2022-12-27 15:15:00,BUY
23,2022-12-27 14:15:00,SELL
24,2022-12-27 14:00:00,BUY
25,2022-12-27 13:45:00,SELL
38,2022-12-27 10:30:00,BUY
41,2022-12-27 09:45:00,SELL


In [84]:
strategy.plot()